In [ ]:
%%capture --no-stderr 
!pip install -q langgraph langchain python-dotenv  langsmith langchain_google_genai langchain_core langchain_community tavily-python

In [4]:
from dotenv import load_dotenv
import os 
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
if not api_key:
    raise ValueError("GEMINI_API_KEY is not set in the .env file.")

print("API key loaded from .env file.")


API key loaded from .env file.


In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings,ChatGoogleGenerativeAI 
from langchain_core.messages import HumanMessage, AIMessage

faq_data = [
    {'question': "How do I track my order?", 'answer': "You can track your order by logging into your account and going to the 'Order History' section."},
    {'question': "What is your return policy?", 'answer': "Our return policy allows returns within 30 days of purchase."},
    {"question": "Can I cancel or modify my order after placing it?","answer": "Yes, you can cancel or modify your order within 24 hours of placing it by contacting our customer support team."},
    {"question": "What should I do if I receive a damaged or incorrect item?", "answer": "If you receive a damaged or incorrect item, please contact our support team immediately with your order details and a photo of the item."},
    {"question": "Do you offer discounts or promotional codes?","answer": "Yes, we regularly offer discounts and promotional codes. Sign up for our newsletter or follow us on social media to stay updated."},
    {"question": "How long does delivery take?",  "answer": "Delivery typically takes 3-7 business days for domestic orders and 7-14 business days for international orders."},
    {"question": "Is my personal information secure?", "answer": "Yes, we use industry-standard encryption and security measures to protect your personal information and ensure safe transactions."}
]


gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001" ,google_api_key=GEMINI_API_KEY)
def faq_data_with_embeddings(faq_data):
    for faq in faq_data:
        faq['embedding'] = gemini_embeddings.embed_query(faq['question'])
    return faq_data


# enriched_faq_data = faq_data_with_embeddings(faq_data) 

In [6]:
user_query = "i want to track my order and what is my name"
query_embedding = gemini_embeddings.embed_query(user_query)

In [7]:
import numpy as np
from typing import List

def find_similar(vectorA:List[int],vectorB:List[int]) -> int :
    cosine_sim = np.dot(vectorA, vectorB) / (np.linalg.norm(vectorA) * np.linalg.norm(vectorB))    
    return cosine_sim 


In [8]:

for index , faq in enumerate(enriched_faq_data):
    sim = find_similar(query_embedding,faq["embedding"])
    enriched_faq_data[index]["cosine_similarity"] = sim


In [9]:
MATCHING_THRESHOLD = 8
filtered_faq = [(faq["question"],faq["answer"]) for faq in enriched_faq_data if faq["cosine_similarity"] >= MATCHING_THRESHOLD]
for question,answer in filtered_faq:
    print(f"Question:{question},Answer:{answer}")
 

In [21]:
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
context = "\n".join([f"Question: {faq['question']}\nAnswer: {faq['answer']}" for faq in faq_data])
prompt_template = PromptTemplate(
    input_variables=["context", "user_query"],
    template="""
        You are a helpful and knowledgeable chatbot for an e-commerce website. Your job is to answer customer questions based on the provided context. If the answer cannot be found in the context, respond politely with: "I’m sorry, I don’t know the answer to that right now."

        ### Context:  
        {context}

        ### User Question:  
        {user_query}

        ### Guidelines for Responses:
        1. Answer based on the provided context.
        2. Be concise, friendly, and professional.
        3. If unsure, say: "I’m sorry, I don’t know the answer to that right now."
        4. Offer help where possible, like checking stock, suggesting products, or explaining policies.

        ### Response:
    """
) 
formatted_prompt = prompt_template.format(context=context, user_query=user_query)
gemini_chat_model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key= GEMINI_API_KEY,
    temperature=0.3,
)
response = gemini_chat_model([HumanMessage(content=formatted_prompt)]) 
print("Answer:", response.content)

Answer: You can track your order by logging into your account and going to the 'Order History' section.  I’m sorry, I don’t know the answer to that right now. (regarding your name).

